### 1 - Import test data

In [1]:
import yfinance as yf
import pandas as pd

dataF = yf.download("EURUSD=X", start="2024-10-1", end="2024-11-1", interval='5M')
dataF.columns = dataF.columns.get_level_values('Price')
dataF.iloc[:,:]
# dataF.Open.iloc

[*********************100%***********************]  1 of 1 completed


Price,Adj Close,Close,High,Low,Open,Volume
Datetime,,,,,,
2024-10-01 00:00:00,1.114455,1.114455,1.114455,1.114455,1.114455,0
2024-10-01 00:05:00,1.114330,1.114330,1.114579,1.114330,1.114455,0
2024-10-01 00:10:00,1.114455,1.114455,1.114455,1.114455,1.114455,0
2024-10-01 00:15:00,1.114579,1.114579,1.114579,1.114455,1.114455,0
2024-10-01 00:20:00,1.114455,1.114455,1.114579,1.114330,1.114579,0
...,...,...,...,...,...,...
2024-10-31 23:35:00,1.088732,1.088732,1.088850,1.088732,1.088732,0
2024-10-31 23:40:00,1.088850,1.088850,1.088850,1.088850,1.088850,0
2024-10-31 23:45:00,1.088732,1.088732,1.088850,1.088732,1.088732,0


### 2 - Define your signal function - Where you put your strat

In [2]:
#Extras
# # ANSI escape sequences for text color
# print("\033[1;31mThis is red text")    # Red text
# print("\033[1;32mThis is green text")  # Green text
# print("\033[1;33mThis is yellow text") # Yellow text
# print("\033[0mThis is default text")   # Reset to default color

def signal_generator(df):#Engulfing pattern
    # Extract scalar values for open and close prices
    # print("signal_generator running")
    open = df.Open.iloc[-1]
    close = df.Close.iloc[-1]
    prevOpen = df.Open.iloc[-2]
    prevClose = df.Close.iloc[-2]
    # print("\033[1;33m"+ "| open: " + str(open) + " | close: " + str(close) + " | prevOpen: " + str(prevOpen) + " | prevClose: " + str(prevClose) + "\033[0m", end="")

    # Bearish Pattern
    if ( (open > close) and (prevOpen < prevClose) and 
        (close < prevOpen) and (open >= prevClose) ):
        # print(" _-Bearish Pattern-_")
        return 1

    # Bullish Pattern
    elif ( (open < close) and (prevOpen > prevClose) and 
          (close > prevOpen) and (open <= prevClose) ):
        # print(" _-Bearish Pattern-_")
        return 2

    # No clear pattern
    else:
        # print(" _-Bearish Pattern-_")
        return 0


#Test dataframe self

# Generate signals
signal = [0]  # First row has no signal
for i in range(1, len(dataF)):
    # Slice two rows for signal generation
    df = dataF.iloc[i-1:i+1]
    signal.append(signal_generator(df))

print("\033[1;32m___Signal generating DONE___\033[0m", )#Set text to green and reset to default

# Assign the signals to the DataFrame
dataF["signal"] = signal

# Check signal distribution
print(dataF["signal"].value_counts())
dataF.iloc[:,:]

#Write current dataF to excel spreadsheet
dataF.to_excel('dataF_1.xlsx',header=True, index=True)


___Signal generating DONE___
signal
0    6356
1      88
2      84
Name: count, dtype: int64


In [3]:
#Count signals on dataF
dataF.signal.value_counts()
#dataF.iloc[:, :]

signal
0    6356
1      88
2      84
Name: count, dtype: int64

### 3 - Connect to the market

In [6]:
import MetaTrader5 as mt5
from apscheduler.schedulers.blocking import BlockingScheduler
from MetaTrader5 import TradeRequest, OrderSendResult, TRADE_ACTION_DEAL
from datetime import datetime

# Connect to MetaTrader 5
if not mt5.initialize(login=98710183, password="Aidan#1947", server="XMGlobal-MT5 5"):
    print("Failed to initialize MT5, error code:", mt5.last_error())
    quit()



# Function to fetch candles
def get_candles(n):
    """
    Fetch the latest `n` candlestick data for EURUSD on the M15 timeframe.
    """
    symbol = "EURUSD"  # Replace with the desired trading pair
    timeframe = mt5.TIMEFRAME_M5  # 5-minute candlesticks

    # Get the latest `n` candles
    rates = mt5.copy_rates_from_pos(symbol, timeframe, 0, n)
    if rates is None:
        print(f"Failed to fetch candles for {symbol}. Error:", mt5.last_error())
        return None

    # Convert to a Pandas DataFrame for easier processing (optional)
    rates_frame = pd.DataFrame(rates)
    rates_frame['time'] = pd.to_datetime(rates_frame['time'], unit='s')
    return rates_frame

# Fetch 3 candles
candles = get_candles(80)
print(candles)
if candles is not None:
    # Example: Check if the opening bid price of each candle is > 1
    for index, candle in candles.iterrows():
        print(float(candle['open']) > 1)


                  time     open     high      low    close  tick_volume  \
0  2024-11-22 17:20:00  1.04157  1.04171  1.04112  1.04144          853   
1  2024-11-22 17:25:00  1.04143  1.04183  1.04102  1.04117          790   
2  2024-11-22 17:30:00  1.04116  1.04120  1.04052  1.04062          780   
3  2024-11-22 17:35:00  1.04062  1.04107  1.04041  1.04085          621   
4  2024-11-22 17:40:00  1.04085  1.04126  1.04057  1.04120          520   
..                 ...      ...      ...      ...      ...          ...   
75 2024-11-22 23:35:00  1.04153  1.04154  1.04135  1.04137           94   
76 2024-11-22 23:40:00  1.04136  1.04144  1.04135  1.04143           76   
77 2024-11-22 23:45:00  1.04143  1.04161  1.04135  1.04135          139   
78 2024-11-22 23:50:00  1.04135  1.04167  1.04135  1.04164           32   
79 2024-11-22 23:55:00  1.04163  1.04192  1.04104  1.04130          167   

    spread  real_volume  
0       17            0  
1       17            0  
2       17           

In [7]:

def trading_job():
    candles = get_candles(3)  # Get 3 most recent candles
    dfstream = pd.DataFrame(columns=['Open', 'Close', 'High', 'Low'])

    # Populate dataframe with candle data
    for i, candle in candles.iterrows():
        dfstream.loc[i, ['Time']] = candle['time']
        dfstream.loc[i, ['Open']] = float(candle['open'])
        dfstream.loc[i, ['Close']] = float(candle['close'])
        dfstream.loc[i, ['High']] = float(candle['high'])
        dfstream.loc[i, ['Low']] = float(candle['low'])

    dfstream['Time'] = pd.to_datetime(dfstream['Time'])
    dfstream['Open'] = dfstream['Open'].astype(float)
    dfstream['Close'] = dfstream['Close'].astype(float)
    dfstream['High'] = dfstream['High'].astype(float)
    dfstream['Low'] = dfstream['Low'].astype(float)

    signal = signal_generator(dfstream.iloc[:-1,:])  # Your signal logic

    # EXECUTING ORDERS

    symbol = "EURUSD"  # Trading pair symbol
    SLTPRatio = 2.0  # Ratio to calculate TP/SL
    previous_candleR = abs(dfstream['High'].iloc[-2] - dfstream['Low'].iloc[-2])

    # Get symbol information
    symbol_info = mt5.symbol_info(symbol)
    print(symbol_info)

    # Example for adjusting SL and TP to avoid invalid stops
    price = mt5.symbol_info_tick(symbol).ask  # Get the current ask price

    # Ensure that the stop levels respect minimum distance
    min_stop_level = symbol_info.trade_stops_level  # Minimum stop level in points
    spread = symbol_info.spread  # Spread in points
    min_stop_distance = max(5, min_stop_level)  # Ensure at least the broker's minimum stop level

    # Calculate Stop Loss and Take Profit levels
    SLBuy = price - previous_candleR - min_stop_distance * symbol_info.point  # 5 points away from the price
    SLSell = price + previous_candleR + min_stop_distance * symbol_info.point

    TPBuy = price + previous_candleR * SLTPRatio + min_stop_distance * symbol_info.point
    TPSell = price - previous_candleR * SLTPRatio - min_stop_distance * symbol_info.point

    # Ensure that SL and TP are in correct direction
    print(f"Calculated SL for Buy: {SLBuy}, TP for Buy: {TPBuy}")
    print(f"Calculated SL for Sell: {SLSell}, TP for Sell: {TPSell}")

    # Optional: Print the dataframe without the last row for clarity
    print(dfstream.iloc[:-1,:])

    # Determine buy/sell based on signal
    if signal == signal:  # Sell signal = 1
        # Create a sell order
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 0.1,  # Lot size (adjust as needed)
            "type": mt5.ORDER_TYPE_SELL,
            "price": mt5.symbol_info_tick(symbol).ask,
            "sl": SLSell,  # Ensure sell SL
            "tp": TPSell,  # Ensure sell TP
            "deviation": 10,
            "magic": 234000,
            "comment": "Python Sell Order",
            "type_filling": mt5.ORDER_FILLING_IOC
        }
        print("\033[1;31mPlacing Sell order: \033[0m", end="\n==============================================================\n")
        result = mt5.order_send(request)
        print(result)

    elif signal == 2:  # Buy signal
        # Create a buy order
        request = {
            "action": mt5.TRADE_ACTION_DEAL,
            "symbol": symbol,
            "volume": 0.1,  # Lot size (adjust as needed)
            "type": mt5.ORDER_TYPE_BUY,
            "price": mt5.symbol_info_tick(symbol).bid,
            "sl": SLBuy,  # Ensure buy SL
            "tp": TPBuy,  # Ensure buy TP
            "deviation": 10,
            "magic": 234000,
            "comment": "Python Buy Order",
            "type_filling": mt5.ORDER_FILLING_IOC
        }
        print("\033[1;32mPlacing Buy order: \033[0m", end="\n==============================================================\n")
        result = mt5.order_send(request)
        print(result, end="\n==============================================================\n")


### 4 - Executing orders automatically with a scheduler

In [37]:

if not mt5.initialize(login=98710183, password="Aidan#1947", server="XMGlobal-MT5 5"):
    print("Failed to initialize MT5, error code:", mt5.last_error())
    quit()
    
# Run the trading job
trading_job()

#Execute orders at what times:

# scheduler = BlockingScheduler() # Used for running jobs at specified intervals. Will keep jobs runnning and schedule jobs accordingly.

# #Used to specify that it should run trading job on monday to friday schedule / Run every hour from 00:00 to 23:00 / run every specified minute: 1,16,31,46 / Start date / timezone
# scheduler.add_job(trading_job, 'cron', day_of_week='mon-fri', hour='00-23', minute='1,6,11,16,21,26,31,36,41,46,51,56', start_date='2022-01-12 12:00:00', timezone='America/Chicago')
# scheduler.start()


SymbolInfo(custom=False, chart_mode=0, select=True, visible=True, session_deals=0, session_buy_orders=0, session_sell_orders=0, volume=0, volumehigh=0, volumelow=0, time=1732309802, digits=5, spread=20, spread_float=True, ticks_bookdepth=0, trade_calc_mode=0, trade_mode=4, start_time=0, expiration_time=0, trade_stops_level=0, trade_freeze_level=0, trade_exemode=2, swap_mode=1, swap_rollover3days=3, margin_hedged_use_leg=False, expiration_mode=15, filling_mode=2, order_mode=127, order_gtc_mode=0, option_mode=0, option_right=0, bid=1.0416, bidhigh=1.0497, bidlow=1.03309, ask=1.0418, askhigh=1.04991, asklow=1.03336, last=0.0, lasthigh=0.0, lastlow=0.0, volume_real=0.0, volumehigh_real=0.0, volumelow_real=0.0, option_strike=0.0, point=1e-05, trade_tick_value=1.0, trade_tick_value_profit=1.0, trade_tick_value_loss=1.0, trade_tick_size=1e-05, trade_contract_size=100000.0, trade_accrued_interest=0.0, trade_face_value=0.0, trade_liquidity_rate=0.0, volume_min=0.01, volume_max=50.0, volume_step

In [14]:
# scheduler.shutdown()
mt5.shutdown()

True